In [1]:
# This file stores the filament related SQL database functions and generating QR code

In [1]:
from PIL import Image
from brother_ql.conversion import convert
from brother_ql.backends.helpers import send
from brother_ql.raster import BrotherQLRaster

deprecation warning: brother_ql.devicedependent is deprecated and will be removed in a future release


In [2]:
import qrcode
import cv2
import urllib
import numpy as np
import psycopg2
import datetime

In [3]:
# QR code related functions

# generate and save a QRcode by input filamentID
def gen_QRcode(filamentid):
    img = qrcode.make(filamentid)
    type(img)
    return img

# read QR code by img
def read_QRcode(img):
    try:
        detect = cv2.QRCodeDetector()
        value, points, straight_qrcode = detect.detectAndDecode(img)
        return value
    except:
        return
    
# read QR code from a octopi camera
# Keep reading until detect QRcode
def scan_QRcode_by_printer(printerid):
    W = True
    while W:
        req = urllib.request.urlopen("http://172.31.1.22"+str(printerid+4)+":8080/?action=snapshot")#
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img = cv2.imdecode(arr, -1)
        result = read_QRcode(img)
        if result != '':
            W = False
            return result

In [4]:
# LabelMaker related functions

# generate a picture with QR code on left and information on right
def generate_label(filamentid, colour, filament_diameter, material, today, price):
    qr = gen_QRcode(str(filamentid))
    ones = np.ones([290, 500])
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1.15
    thickness = 2

#     now = datetime.datetime.now().strftime("%Y-%m-%d")
    ones = cv2.putText(ones, 'Filament ID : %s' %str(filamentid), (0, 70), font, 
                       fontScale, 0, thickness, cv2.LINE_AA)
    ones = cv2.putText(ones, '%s         %s' %(colour, material), (0, 125), font, 
                       fontScale, 0, thickness, cv2.LINE_AA)
    ones = cv2.putText(ones, '%s%s      %s%s' %(filament_diameter, 'mm', '$', price), (0, 180), font, 
                       fontScale, 0, thickness, cv2.LINE_AA)
    ones = cv2.putText(ones, 'Open Date: %s' %today, (0, 235), font, 
                       fontScale, 0, thickness, cv2.LINE_AA)
    pic = np.concatenate([qr, ones], axis=1)
    return pic*255

# Original code from brother_ql
def send_to_labelmaker(im):
    im = Image.fromarray(im)
    backend = 'pyusb'    # 'pyusb', 'linux_kernal', 'network'
    model = 'QL-700' # your printer model.
    printer = 'usb://0x04f9:0x2042'  # Get these values from the Windows usb driver filter.  Linux/Raspberry Pi uses '/dev/usb/lp0'.
    qlr = BrotherQLRaster(model)
    qlr.exception_on_warning = True

    instructions = convert(
            qlr=qlr, 
            images=[im],    #  Takes a list of file names or PIL objects.
            label='62', 
            rotate='0',    # 'Auto', '0', '90', '270'
            threshold=70.0,    # Black and white threshold in percent.
            dither=False, 
            compress=False, 
            red=False,    # Only True if using Red/Black 62 mm label tape.
            dpi_600=False, 
            hq=True,    # False for low quality.
            cut=True

    )

    send(instructions=instructions, printer_identifier=printer, backend_identifier=backend, blocking=True)

In [31]:
# im = generate_label(1, 'Green', '1.75', 'PLA+', '0000-11-22', '26.9')
# cv2.imwrite('test.png', im)
# send_to_labelmaker(im)

True

In [15]:
# 原始copy过来的brother_ql的example code

# im = Image.open('try.png')
# im.resize((10, 10)) 

# backend = 'pyusb'    # 'pyusb', 'linux_kernal', 'network'
# model = 'QL-700' # your printer model.
# printer = 'usb://0x04f9:0x2042'  #04f9&pid 2042&rev 0100
# # Get these values from the Windows usb driver filter.  Linux/Raspberry Pi uses '/dev/usb/lp0'.
# # printer = 'usbstor.inf:392c3d533eff3360:USBSTOR_BULK.NT:10.0.19041.1949:USB\Class_08&SubClass_06&Prot_50'  
# qlr = BrotherQLRaster(model)
# qlr.exception_on_warning = True

# instructions = convert(

#         qlr=qlr, 
#         images=[im],    #  Takes a list of file names or PIL objects.
#         label='62', 
#         rotate='0',    # 'Auto', '0', '90', '270'
#         threshold=70.0,    # Black and white threshold in percent.
#         dither=False, 
#         compress=False, 
#         red=False,    # Only True if using Red/Black 62 mm label tape.
#         dpi_600=False, 
#         hq=True,    # False for low quality.
#         cut=True

# )

# send(instructions=instructions, printer_identifier=printer, backend_identifier=backend, blocking=True)

Need to resize the image...


{'instructions_sent': True,
 'outcome': 'printed',
 'printer_state': {'status_type': 'Phase change',
  'phase_type': 'Waiting to receive',
  'media_type': 'Continuous length tape',
  'media_width': 62,
  'media_length': 0,
  'errors': []},
 'did_print': True,
 'ready_for_next_job': True}

In [5]:
connection = psycopg2.connect(database='DTLSQLV2', user='postgres', password='4321', host='localhost')

In [6]:
cursor=connection.cursor()

In [7]:
from database import CursorFromConnectionPool
from database import Database

In [8]:
Database.initialise(database="DTLSQLV2", user="postgres", password="4321", host="localhost")

In [9]:
# no filamentID required for input, generate automatically when input new filament info
class Filament:
    def __init__(self, material, colour, filament_diameter, current_loc, price, open_date, brand, left_weight):
        self.material = material
        self.colour = colour
        self.filament_diameter = filament_diameter
        self.current_loc = current_loc
        self.price = price
        self.open_date = open_date
        self.brand = brand
        self.left_weight = left_weight
    
    @classmethod
    # when take a filament off from the printer and update its weight to database
    # should scan the QR code first to get the filament id
    def update_filament_weight(cls, filamentid, new_weight):
        with CursorFromConnectionPool() as cursor:
            cursor.execute('UPDATE filament SET left_weight=%s WHERE filamentid=%s;', (new_weight, filamentid,))
    @classmethod
    # when take a filament off from the printer and update its weight to database
    # should scan the QR code first to get the filament id
    def update_filament_loc(cls, filamentid, new_loc):
        with CursorFromConnectionPool() as cursor:
            cursor.execute('UPDATE filament SET current_loc=%s WHERE filamentid=%s;', (new_loc, filamentid,))
    @classmethod
    # add a new roll of filament to database
    # automatically print QR code from labelmaker
    def add_new_filament(cls, material, colour, filament_diameter, current_loc, price, brand, left_weight): 
        with CursorFromConnectionPool() as cursor:
            today = datetime.datetime.now().strftime("%Y-%m-%d")
            cursor.execute('INSERT INTO filament (material, colour, filament_diameter, current_loc, \
            price, open_date, brand, left_weight) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)',
                            (material, colour, filament_diameter, current_loc, price, today, brand, left_weight,))
            cursor.execute('SELECT filamentid FROM filament ORDER BY filamentid DESC')
            filamentid = cursor.fetchone()[0]
            im = generate_label(filamentid, colour, filament_diameter, material, today, price)
            send_to_labelmaker(im)
#----------------------under construction from here downwards--------------------------
    @classmethod
    # 新增一个filament 随便写来不trigger label maker用python自动生成虚假数据的
    def add_new_filament_no_label(cls, material, colour, filament_diameter, current_loc, price, date, brand, left_weight): 
        with CursorFromConnectionPool() as cursor:
            cursor.execute('INSERT INTO filament (material, colour, filament_diameter, current_loc, \
            price, open_date, brand, left_weight) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)',
                            (material, colour, filament_diameter, current_loc, price, date, brand, left_weight,))
    

In [10]:
# 用来随便generate日期的
from random import randrange
from datetime import timedelta

def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)
d1 = datetime.datetime.strptime('2021-1-1', "%Y-%m-%d")
d2 = datetime.datetime.strptime('2022-10-1', "%Y-%m-%d")

print(random_date(d1, d2).strftime("%Y-%m-%d"))

2022-06-19


In [64]:
Filament.add_new_filament_no_label('PLA+','white', '1.75', 'notexist', '26.9', random_date(d1, d2).strftime("%Y-%m-%d"), '3DFILLIES', '0')

In [108]:
# scan_QRcode_by_printer(4)

In [11]:
class Printer:
    def __init__(self, printerid, status, start_time, filamentid):
        self.printerid = printerid
        self.status = status
        self.start_time = start_time
        self.filamentid = filamentid
    @classmethod
    # 加入一个新的printer
    def add_new_printer(cls, printerid, status, filamentid): 
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with CursorFromConnectionPool() as cursor:
            cursor.execute('INSERT INTO printer_tracking (printerid, status, start_time, filamentid) \
            VALUES (%s, %s, %s, %s)', (printerid, status, now, filamentid))
    @classmethod
    # 已经有了最初的信息要更新printer的状态 只需要输入printerid和新的状态 时间自动抓取现在 filament自动抓取当前的filament
    def update_status_by_printer(cls, printerid, status): 
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with CursorFromConnectionPool() as cursor:
            cursor.execute('SELECT filamentid FROM printer_tracking WHERE printerid=%s ORDER BY start_time DESC;', \
                           (printerid,))
            filamentid = cursor.fetchone()
            cursor.execute('INSERT INTO printer_tracking (printerid, status, start_time, filamentid) \
            VALUES (%s, %s, %s, %s)', (printerid, status, now, filamentid))
    @classmethod
    # 更改filament 这里不需要check状态是maintaning check maintaning的步骤从OPCUA层面实现
    # 没有filament的时候filamentid的位置输入None
    def load_filamentid_by_printer(cls, printerid, filamentid):
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with CursorFromConnectionPool() as cursor:
            cursor.execute('SELECT status FROM printer_tracking WHERE printerid=%s ORDER BY start_time DESC;', \
                           (printerid,))
            status = cursor.fetchone()
            cursor.execute('INSERT INTO printer_tracking (printerid, status, start_time, filamentid) \
            VALUES (%s, %s, %s, %s)', (printerid, status, now, filamentid))
#----------------------under construction from here downwards--------------------------

    
    
#     @classmethod
#     # printer change status, update its new status to database
#     def update_status_by_printer(cls, printerid, new_status):
#         with CursorFromConnectionPool() as cursor:
#             cursor.execute('UPDATE printer SET status=%s WHERE printerid=%s;', (new_status, printerid,))
#     @classmethod
#     # unload filament with scanning
#     def unload_filament_with_scan_by_printer(cls, printerid):
#         with CursorFromConnectionPool() as cursor:
#             cursor.execute('SELECT status, filamentid FROM printer WHERE printerid=%s;', \
#                            (printerid,))
#             status, filamentid = cursor.fetchone()
#             if status=='changing_filament':
#                 if filamentid != None:
#                     cursor.execute('UPDATE printer SET filamentid=NULL WHERE printerid=%s;', (printerid,))
#                     old_filament = scan_QRcode_by_printer(printerid)
#                     print('Filament%s unloaded from Printer%s successfully' %(str(old_filament), str(printerid)))
#                 else:
#                     print('Printer%s is not loaded with any filament' %(str(printerid)))
#             else:
#                 print('Please set Printer%s status to "changing_filament" to change filament' %(str(printerid)))
                
#     @classmethod
#     # load filament with scanning
#     def load_filament_with_scan_by_printer(cls, printerid):
#         with CursorFromConnectionPool() as cursor:
#             cursor.execute('SELECT status, filamentid FROM printer WHERE printerid=%s;', \
#                            (printerid,))
#             status, filamentid = cursor.fetchone()
#             if status=='changing_filament':
#                 if filamentid != None:
#                     print('Please unload the current filament from Printer%s first' %(str(printerid)))
#                 else:
#                     new_filament = scan_QRcode_by_printer(printerid)
#                     cursor.execute('UPDATE printer SET filamentid=%s WHERE printerid=%s;', (new_filament, printerid,))
#                     print('Filament%s has been loaded on Printer%s successfully' %(str(new_filament), str(printerid))) 
#             else:
#                 print('Please set Printer%s status to "changing_filament" to change filament' %(str(printerid)))
#----------------------under construction from here downwards--------------------------

In [12]:
# Printer.update_status_by_printer('1', 'operational')

In [58]:
class Printing:
    def __init__(self, assigned_printerid, start_time, finish_time, snapshot_path, print_result):
        self.assigned_printerid = assigned_printerid
        self.start_time = start_time
        self.finish_time = finish_time
        self.snapshot_path = snapshot_path
        self.print_result = print_result
    @classmethod
    # 开始一个print job 自动发一个partid
    def start_part_by_printerid(cls, printerid): 
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with CursorFromConnectionPool() as cursor:
            cursor.execute('INSERT INTO printing (assigned_printerid, start_time) VALUES (%s, %s)',
                            (printerid, now,))
    @classmethod
    def snapshot_by_printer(cls, printerid):
        req = urllib.request.urlopen("http://172.31.1.22"+str(int(printerid)+4)+":8080/?action=snapshot")
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img = cv2.imdecode(arr, -1)
        return img
    @classmethod
    # 结束一个print job 只给定printerid 程序自己找该printer现在在print哪个partid
    def finish_part_by_printerid(cls, printerid):
        with CursorFromConnectionPool() as cursor:
            cursor.execute('SELECT partid FROM printing WHERE assigned_printerid=%s AND finish_time IS NULL ORDER BY start_time DESC;', 
                           (printerid,))
            if cursor.fetchone() == None:
                print('Printer'+str(printerid)+' does not have unfinished work')
            else:
                now = datetime.datetime.now()
                img = Printing.snapshot_by_printer(printerid)
                path = 'snapshots/Printer'+str(printerid)+'_'+str(now.strftime("%Y-%m-%d_%H-%M-%S"))+'.png'
#                 print(path)
                cv2.imwrite(path, img)
                now = now.strftime("%Y-%m-%d %H:%M:%S")
                cursor.execute('SELECT partid FROM printing WHERE assigned_printerid=%s AND finish_time IS NULL ORDER BY start_time DESC;', 
                           (printerid,))
                partid = cursor.fetchone()[0]
                cursor.execute('UPDATE printing SET finish_time=%s, snapshot_path=%s, print_result=%s WHERE partid=%s;', 
                               (now, path, 'finished', partid,))
#----------------------under construction from here downwards--------------------------

    @classmethod
    # 取消一个print job 只给定printerid 程序自己找该printer现在在print哪个partid
    def cancel_part_by_printerid(cls, printerid):
        with CursorFromConnectionPool() as cursor:
            cursor.execute('SELECT partid FROM printing WHERE assigned_printerid=%s AND finish_time IS NULL ORDER BY start_time DESC;', 
                           (printerid,))
            if cursor.fetchone() == None:
                print('Printer'+str(printerid)+' does not have unfinished work')
            else:
                now = datetime.datetime.now()
                img = Printing.snapshot_by_printer(printerid)
                path = 'snapshots/Printer'+str(printerid)+'_'+str(now.strftime("%Y-%m-%d_%H-%M-%S"))+'.png'
#                 print(path)
                cv2.imwrite(path, img)
                now = now.strftime("%Y-%m-%d %H:%M:%S")
                cursor.execute('SELECT partid FROM printing WHERE assigned_printerid=%s AND finish_time IS NULL ORDER BY start_time DESC;', 
                           (printerid,))
                partid = cursor.fetchone()[0]
                cursor.execute('UPDATE printing SET finish_time=%s, snapshot_path=%s, print_result=%s WHERE partid=%s;', 
                               (now, path, 'cancelled', partid,))

#     @classmethod
#     def load_from_db_by_part(cls, partid):
#         with CursorFromConnectionPool() as cursor:
#             cursor.execute('SELECT * FROM printing WHERE partid=%s', (partid,))
#             print_data = cursor.fetchone()
#             return cls(status=print_data[1], assigned_printerid=print_data[2], initiate_time=print_data[3], 
#                        start_time=print_data[4], finish_time=print_data[5], cooldown_time=print_data[6],
#                        pickup_time=print_data[7], cleanup_time=print_data[8], snapshot_path=print_data[9],
#                        filamentid=print_data[10], used_filament=print_data[11],)
    
#     @classmethod
#     # get current print job on printerX. either waiting or printing. Each printer can have only one job at a time
#     def get_current_partid_by_printer(cls, printerid):
#         with CursorFromConnectionPool() as cursor:
#             cursor.execute('SELECT partid FROM printing WHERE assigned_printerid=%s AND (status=%s OR status=%s\
#              OR status=%s OR status=%s)', (printerid, 'waiting','printing', 'cooling', 'onbed',))
#             return cursor.fetchone()[0]


In [59]:
Printing.cancel_part_by_printerid('2')

snapshots/Printer2_2022-10-25_12-22-45.png


In [71]:
img = cv2.imread('qr1.png')
read_QRcode(img)

'1'

In [11]:
# Keenan写的用来检测labelmaker的usbid的code
#!/usr/bin/python
import sys
import usb.core
# find USB devices
dev = usb.core.find(find_all=True)
# loop through devices, printing vendor and product ids in decimal and hex
for cfg in dev:
  sys.stdout.write('Decimal VendorID=' + str(cfg.idVendor) + ' & ProductID=' + str(cfg.idProduct) + '\n')
  sys.stdout.write('Hexadecimal VendorID=' + hex(cfg.idVendor) + ' & ProductID=' + hex(cfg.idProduct) + '\n\n')

Decimal VendorID=1273 & ProductID=8258
Hexadecimal VendorID=0x4f9 & ProductID=0x2042

